In [0]:
from pyspark import *
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains, struct, collect_list, row_number

In [0]:
%fs ls /Volumes/big_data_ii_2025/spark_examples/spark_data

path,name,size,modificationTime
dbfs:/Volumes/big_data_ii_2025/spark_examples/spark_data/PADRON_COMPLETO.csv,PADRON_COMPLETO.csv,437542254,1750034299000
dbfs:/Volumes/big_data_ii_2025/spark_examples/spark_data/distelec.csv,distelec.csv,175692,1750034273000
dbfs:/Volumes/big_data_ii_2025/spark_examples/spark_data/movies.csv,movies.csv,4192335,1750034274000
dbfs:/Volumes/big_data_ii_2025/spark_examples/spark_data/padron_limpio.csv/,padron_limpio.csv/,0,1750039677659
dbfs:/Volumes/big_data_ii_2025/spark_examples/spark_data/por-ciclo-2016-2018.csv,por-ciclo-2016-2018.csv,904931,1750034273000
dbfs:/Volumes/big_data_ii_2025/spark_examples/spark_data/ratings_full.csv,ratings_full.csv,933898879,1750034299000
dbfs:/Volumes/big_data_ii_2025/spark_examples/spark_data/tags.csv,tags.csv,85361813,1750034296000


# Manejo y Optimización de Particiones en Spark

En este notebook aprenderás a:
- Visualizar y modificar el número de particiones de un DataFrame.
- Usar las funciones `repartition` y `coalesce`.
- Guardar datos con particionado por columna y medir el impacto en el rendimiento.
Trabajaremos con el dataset MovieLens.


In [0]:
# Cargar datos MovieLens (ratings.csv) suponiendo que está en /dbfs/FileStore/movielens/ratings.csv
# Tabla Ratings
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("/Volumes/big_data_ii_2025/spark_examples/spark_data/ratings_full.csv")\
    .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')



In [0]:
ratingsDf.printSchema()


root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: decimal(2,1) (nullable = true)
 |-- date: string (nullable = true)



In [0]:
print(f"Ratings count: {ratingsDf.count():,.0f}")



Ratings count: 33,832,162


In [0]:
# Ver número de particiones
print("Particiones iniciales:", ratingsDf.rdd.getNumPartitions())



---------------------------------------------------------------------------
PySparkNotImplementedError                Traceback (most recent call last)
File <command-6212165207225491>, line 2
      1 # Ver número de particiones
----> 2 print("Particiones iniciales:", ratingsDf.rdd.getNumPartitions())

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/dataframe.py:2432, in DataFrame.rdd(self)
   2430 @property
   2431 def rdd(self) -> "RDD[Row]":
-> 2432     raise PySparkNotImplementedError(
   2433         error_class="NOT_IMPLEMENTED",
   2434         message_parameters={"feature": "rdd"},
   2435     )

PySparkNotImplementedError: [NOT_IMPLEMENTED] Using custom code using PySpark RDDs is not allowed on serverless compute. We suggest using mapInPandas or mapInArrow for the most common use cases. For more details on compatibility and limitations, check: https://docs.databricks.com/release-notes/serverless.html#limitations

In [0]:
%time
# Cambiar el número de particiones a 16
df_repart = ratingsDf.repartition(16)
print("Después de repartition(16):", df_repart.rdd.getNumPartitions())



Después de repartition(16): 16


In [0]:
%time
# Reducir a 2 particiones con coalesce
df_coalesce = df_repart.coalesce(2)
print("Después de coalesce(2):", df_coalesce.rdd.getNumPartitions())



Después de coalesce(2): 2


In [0]:
%time
# Guardar datos particionando por 'rating'
import time
start = time.time()
output_path = "/Volumes/big_data_ii_2025/spark_examples/spark_data/rating_by_rating"
ratingsDf.write.mode("overwrite").partitionBy("rating").parquet(output_path)
print("Tiempo:", time.time() - start, "segundos")


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs
Tiempo: 104.14708518981934 segundos


In [0]:
%%sh
ls -las /Volumes/big_data_ii_2025/spark_examples/spark_data/rating_by_rating/

total 8
4 drwxr-xr-x 2 root root 4096 Jun  8 20:18 .
4 drwxrwxrwt 1 root root 4096 Jun  8 20:18 ..


In [0]:
print("Particiones originales:", ratingsDf.rdd.getNumPartitions())



Particiones originales: 8


In [0]:
def medir_tiempo_particiones(df, num_particiones):
    df_mod = df.repartition(num_particiones)
    print(f"\nUsando {num_particiones} particiones:")
    start = time.time()
    # Operación costosa: groupBy + count
    resultado = df_mod.groupBy("movieId").count().collect()
    print("Tiempo:", round(time.time() - start, 2), "segundos")



In [0]:
for n in [2, 4, 8, 16, 32]:
    medir_tiempo_particiones(ratingsDf, n)


Usando 2 particiones:
Tiempo: 63.93 segundos

Usando 4 particiones:
Tiempo: 63.58 segundos

Usando 8 particiones:
Tiempo: 64.058 segundos

Usando 16 particiones:
Tiempo: 65.2 segundos

Usando 32 particiones:
Tiempo: 66.018 segundos
